In [1]:
import sys
sys.path.insert(0,'py_files')
from model_recurrent import *

C:\Users\nicol\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load and pad images

In [2]:
# Load a set of image
root_dir = "../Data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) 
print("Loading " + str(n) + " images")
imgs_original = [load_image(image_dir + files[i]) for i in range(n)]
imgs = [load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
gt_imgs_not_padded = [load_image(gt_dir + files[i]) for i in range(n)]

print('Padding images using pad of: ', pad_rotate_size)
imgs = padding_imgs(np.array(imgs),pad_rotate_size)
gt_imgs = padding_GT(np.array(gt_imgs),pad_rotate_size)
print(imgs.shape)
print(gt_imgs.shape)

Loading 100 images
Loading 100 images
Padding images using pad of:  47
(100, 494, 494, 3)
(100, 494, 494)


# Training

# split in validation + train: TODO, random

In [ ]:
X_train = imgs
Y_train = gt_imgs
model = train(X_train,Y_train)

Training set shape:  (100, 494, 494, 3)
Batch_size: 250 
Steps per epoch: 125 



py_files\model_recurrent.py:175: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  callbacks=[lr_callback, stop_callback])
py_files\model_recurrent.py:175: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=125, verbose=1, callbacks=[<keras.ca..., epochs=45)`
  callbacks=[lr_callback, stop_callback])


Epoch 1/45
125/125 [==============================] - 563s 5s/step - loss: 0.5617 - acc: 0.7472 - f1_score: 0.7472
Epoch 2/45
125/125 [==============================] - 562s 4s/step - loss: 0.5265 - acc: 0.7442 - f1_score: 0.7442
Epoch 3/45
125/125 [==============================] - 575s 5s/step - loss: 0.4992 - acc: 0.7606 - f1_score: 0.7606
Epoch 4/45
125/125 [==============================] - 571s 5s/step - loss: 0.4601 - acc: 0.7801 - f1_score: 0.7801
Epoch 5/45
125/125 [==============================] - 550s 4s/step - loss: 0.4313 - acc: 0.7984 - f1_score: 0.7984
Epoch 6/45
125/125 [==============================] - 545s 4s/step - loss: 0.3771 - acc: 0.8322 - f1_score: 0.8322
Epoch 7/45
125/125 [==============================] - 548s 4s/step - loss: 0.3629 - acc: 0.8399 - f1_score: 0.8399
Epoch 8/45
 42/125 [=========>....................] - ETA: 6:21 - loss: 0.3425 - acc: 0.8514 - f1_score: 0.8514

# if we want to go on with the training ...

In [ ]:
# X_train = imgs
# Y_train = gt_imgs
# model, lr_callback, stop_callback = create_model()
# model.load_weights('NicolaWeights0612')

# np.random.seed(3) # We don't want to take the same data
    
# try:
#     model.fit_generator(generate_minibatch(X_train,Y_train),
#                             steps_per_epoch=steps_per_epoch,
#                             nb_epoch=3,
#                             verbose=1,
#                             callbacks=[lr_callback, stop_callback])
# except KeyboardInterrupt:
#     print('\n\nKeyboard interruption!\n\n')
#     pass
# model.save_weights('NicolaWeights2')

# Check if on the train set we have good results

In [ ]:
IDX = 1
X = imgs_to_windows(imgs_original,400,patch_size,input_size)
patches_idx = X[625*IDX:625*(IDX+1)]

Z_idx = model.predict(patches_idx)
labels_idx = (Z_idx[:,0] < Z_idx[:,1]) * 1 
predicted_image = label_to_img(400,400,16,16,labels_idx)
temp = concatenate_images(imgs_original[IDX],predicted_image)
plt.imshow(concatenate_images(temp, gt_imgs_not_padded[IDX]))

# validation

In [ ]:
# Recover the weights without training everything again
model,_,_ = create_model()
model.load_weights('model_2b_Weights')

N_valid = 3
N = 100
X_valid = imgs_original[N-N_valid:]
X_valid = padding_imgs(np.array(X_valid),pad_size)

#create the input for the model
val_inputs = imgs_to_windows(X_valid,400,patch_size,input_size)
print(val_inputs.shape)

In [ ]:
#get the true value of the labels
Y_valid = gt_imgs_not_padded[N-N_valid:]
val_gt_patches = [img_crop(Y_valid[i], patch_size, patch_size) for i in range(N_valid)]
val_gt_patches =  np.asarray([val_gt_patches[i][j] for i in range(len(val_gt_patches)) for j in range(len(val_gt_patches[i]))])
val_true_labels = np.asarray([value_to_class(np.mean(val_gt_patches[i])) for i in range(len(val_gt_patches))])
print(val_true_labels.shape)


In [ ]:
#predict
val_prediction = model.predict(val_inputs)
print(val_prediction[0:3])
# compute the metrics


In [ ]:
from sklearn.metrics import f1_score
val_predicted_labels = ((val_prediction[:,0] < val_prediction[:,1]) * 1 ).flatten()
val_accuracy = np.sum(np.abs(val_predicted_labels - val_true_labels))/val_true_labels.shape[0]
print('Accuracy on validation set is: ', val_accuracy)
print('F1 score on validation set is:', f1_score(val_true_labels,val_predicted_labels))

# predict on test

In [ ]:
#load images and pad them
test_images = np.asarray(pick_test_images())

test_images = padding_imgs(np.array(test_images),pad_size)

test_images_not_padded = np.asarray(pick_test_images())

In [ ]:
#prepare the input for the prediction
test_inputs = imgs_to_windows(test_images,608,patch_size,input_size)
print(test_inputs.shape)

In [ ]:
#recover the model
model, _,_ = create_model()
model.load_weights('model_2b_Weights')

In [ ]:
#predict
test_prediction = model.predict(test_inputs)
print(test_prediction[0:3])
test_predicted_labels = ((test_prediction[:,0] < test_prediction[:,1]) * 1 ).flatten()

In [ ]:
test_labels = test_predicted_labels.reshape(50,-1)
print('Every row contains the labels of one image')
print(test_labels.shape)

In [ ]:

#check the prediction on the image IDX
IDX = 18
im = label_to_img(608, 608, 16, 16, test_labels[IDX])
plt.figure(figsize=[20,30])
new_img = make_img_overlay(test_images_not_padded[IDX], im)
plt.imshow(new_img)

In [ ]:
MY_masks_to_submission(SubmissionName, test_labels)

# Save the prediction

In [ ]:
import pickle
# Suppose that we create some patches
predicted_patches = test_predicted_labels
print('Predicted patches of size: ', predicted_patches.shape)


# Choose a name for the objects that we are going to save
name = 'prediction1212.pkl'

# Saving the objects:
with open(name, 'wb') as f:
    pickle.dump(predicted_patches, f)
print('Saved in ', name)

# Getting back the objects:
with open(name, 'rb') as f: 
    recovered_patches = pickle.load(f)
print('Recovered!')